# Questionnaire

<img src="img/questionnaire_diagram.png" width=500px/>

Questionnaires rely on 4 classes:

* **Question**: A question to be addressed to the user. Receives user input, parses it and sends to an action;
* **Action**: Processes user input. Results are used to update the context;
* **Context**: Stores user collected data and state. Controls the questions flow in the DAG, gets next question and orchestrates the response.
* **Response**: Formats and sends a response to the user. 

In [125]:
class Question:
    def __init__(self, name, **attr):
        self.name=name
        self.attr=attr
        
        self.prompt = attr.get("prompt")
        self.options = attr.get("options")
        self.options_text = attr.get("options_text")
        self.next = attr.get("next")
        self.user_input_type = attr.get("input_type")
        
    def getOptions(self):
        """
        Returns a dictionary mapping choices (int) to their respective texts.
        """
        if self.options is None: return None
        return { o:t for o,t in zip(self.options, self.options_text) }
    
    def saveUserInput(self, inpt):
        """
        Saves user response (input) to the question.
        input type: 'text','number','choice'
        """
        self.user_input = { 'text':str, 
                            'number':float, 
                            'choice':int }[self.user_input_type]( inpt )
        
    def isFinalQuestion(self):
        return True if self.next is None else False
    
    def checkUserInput(self):
        return self.user_input
    
    def getNextQuestionName(self):
        """
        Checks user input to determine the next question in the graph.
        """
        if len(self.next)>1:
            return self.next[ self.options.index(self.checkUserInput()) ]
        else:
            return self.next[0]
        
    def promptMessage(self):
        """
        Properly formats a message to be prompted to the user.
        """
        if self.user_input_type=='choice':
            return f"{self.prompt} {self.getOptions()}"
        else:
            return self.prompt
        
    
    
class Questionnaire:
    """
    Stores a graph of questions and runs it; keeps track of interactions using a context dictionary.
    """
    def __init__(self, questionnaire_dict=None):
        """
        Questionnaire dict to be used to build the instance
        """
        self.context = dict()
        
        if questionnaire_dict:
            self.questions = { qname: Question(qname,**qattrs) for qname, qattrs in questionnaire_dict.items() }

    def run(self, initialQuestion='q1'):
        currentQuestion = self.questions.get(initialQuestion)
        finish=False
        
        while not finish:
                
            self.context['current_question'] = currentQuestion.name

            response = input(currentQuestion.promptMessage())
            currentQuestion.saveUserInput(response)
            self.context[currentQuestion.name] = currentQuestion.checkUserInput()
            
            if not currentQuestion.isFinalQuestion():
                currentQuestion = self.questions.get(currentQuestion.getNextQuestionName())
            else:
                finish=True
            
        return self.context

One way to build and store a questionnaire is by using a JSON-like strucutre (dict in Python):

In [118]:
questionnaire = {
    'q1': {
        "prompt": "Qual seu nome?",
        "input_type": "text",
        "next":['q2']
          },
    'q2': {
        "prompt": "Qual sua idade?",
        "input_type": "number",
        "next":['q3']
    },
    'q3': {
        "prompt": "O solo está degradado?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": ["não","sim"],
        "next": ['q4']
    },
    'q4': {
        "prompt": "Tem pecuária na vizinhança?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": [ "não", "sim"],
        "next": ['q5']
    },
    'q5': {
        "prompt": "Tem risco de incêndio?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": [ "não", "sim"],
        "next": ['q6']
    },
    'q6': {
        "prompt": "Qual o potencial de regeneração natural?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": [ "baixo potencial", "alto potencial"],
        "next": ['q7','q8']
    },
    'q7':{
        "prompt": "Prefere plantar mudas ou semear?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": ["plantar mudas", "semear"],
        "next": ['q9', 'q10']
    },
    'q8':{
        "prompt": "Prefere manejar a regeneração natural ou não?",
        "input_type": "choice",
        "options": [0,1],
        "options_text": ["não manejar", "manejar"]
    },
    'q9':{
        "prompt": "Que tipo de muda?",
        "input_type": "text"
    },
    'q10':{
        "prompt": "Que sementes?",
        "input_type": "text"
    }
}

Let's test it!

In [127]:
q = Questionnaire(questionnaire)
q.run()

Qual seu nome?Pedro
Qual sua idade?0
O solo está degradado? {0: 'não', 1: 'sim'}0
Tem pecuária na vizinhança? {0: 'não', 1: 'sim'}0
Tem risco de incêndio? {0: 'não', 1: 'sim'}0
Qual o potencial de regeneração natural? {0: 'baixo potencial', 1: 'alto potencial'}0
Prefere plantar mudas ou semear? {0: 'plantar mudas', 1: 'semear'}0
Que tipo de muda?alguma


{'current_question': 'q9',
 'q1': 'Pedro',
 'q2': 0.0,
 'q3': 0,
 'q4': 0,
 'q5': 0,
 'q6': 0,
 'q7': 0,
 'q9': 'alguma'}